<a href="https://colab.research.google.com/github/tr3nt-tayl0r/CIMIS/blob/main/cimis_corrections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone into github repo locally

In [17]:
!git clone https://github.com/tr3nt-tayl0r/CIMIS.git

fatal: destination path 'CIMIS' already exists and is not an empty directory.


In [18]:
import glob
import os
import sys
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import json
import requests
from tables import NaturalNameWarning
import warnings
warnings.filterwarnings('ignore',category = NaturalNameWarning)
verbose = False
import datetime
import matplotlib.pyplot as plt
from google.colab import userdata

In [4]:
def fix_col_names(df):
  '''corrects column names from CIMIS based on dictionary'''
  rename_dict = {'DayAirTmpMin.Value':'Tmin',
               'DayAirTmpMax.Value':'Tmax',
               'DayDewPnt.Value':'Tdew',
               'DayAirTmpAvg.Value':'Tavg',
               'DayEto.Value':'ETo',
               'DayRelHumMin.Value':'RHmin',
               'DayRelHumAvg.Value':'RHavg',
               'DayPrecip.Value':'Pr'}
  df.rename(columns=rename_dict, inplace=True)
  dfout = df[['Date','Tmin','Tmax','Tdew','Tavg','ETo','Pr','RHmin','RHavg']]
  dfout.set_index(pd.to_datetime(dfout.Date),inplace=True)
  return dfout

def corr_nref(df,tminNRef='Tmin',tdewNRef='Tdew',tmaxNRef='Tmax',bT=True):
    df['dT']=df[tminNRef]-df[tdewNRef]
    if bT==True:
        df['bT']=0.3
    dfAI = df[['Pr','ETo']].dropna(axis=0)
    AI = dfAI['Pr'].mean()/df['ETo'].mean()
    print('\t\tAI = '+str(np.round(AI,2)))

    if AI<=0.05:
        aT=5
    elif (AI >0.05) & (AI<=0.2):
        aT=2.5
    elif (AI >0.2) & (AI<=0.5):
        aT=1.5
    elif (AI >0.5) & (AI<=0.65):
        aT=0.5
    else:
        aT=0

    df['aT']=aT

    # Might need to add a line only to correct if dT > aT from the logic statements above
    for dT in df['dT']:
      if dT > aT:
        df[tmaxNRef+'_corr2']= df[tmaxNRef]-df.bT*(df.dT-df.aT) #(2.13)
        df[tminNRef+'_corr2']= df[tminNRef]-df.bT*(df.dT-df.aT) #(2.14)
        df[tdewNRef+'_corr2']= df[tdewNRef]+(1.0-df.bT)*(df.dT-df.aT) #(2.15)]
      else:
        df[tmaxNRef+'_corr2']= df[tmaxNRef]
        df[tminNRef+'_corr2']= df[tminNRef]
        df[tdewNRef+'_corr2']= df[tdewNRef]
    df['AI']= AI
    # df.set_index(df.Date,inplace=True)
    return df

In [24]:
# This cell is grabbing data from all stations via the CIMIS API. There are limits to how big of a date range you can obtain at one time

import requests
import json
import pandas as pd

#daily by station number
#http://et.water.ca.gov/api/data?appKey=46c39c6d-4d71-464c-a20f-a5c7d21bba8e&targets=5&startDate=2024-01-01&endDate=2024-06-06&dataItems=day-eto,day-precip,day-sol-rad-avg,day-vap-pres-avg,day-air-tmp-max,day-air-tmp-min,day-air-tmp-avg,day-rel-hum-max,day-rel-hum-min,day-rel-hum-avg,day-dew-pnt,day-wind-spd-avg,day-wind-run,day-soil-tmp-avg&unitOfMeasure=M

cimis_api = "http://et.water.ca.gov/api"
api_key = userdata.get('cimis_key_gmail')
start_date = "2024-01-01"
end_date = "2024-06-19"
data_items = '''day-eto,day-precip,day-sol-rad-avg,
                day-vap-pres-avg,day-air-tmp-max,day-air-tmp-min,
                day-air-tmp-avg,day-rel-hum-max,day-rel-hum-min,
                day-rel-hum-avg,day-dew-pnt,day-wind-spd-avg,
                day-wind-run,day-soil-tmp-avg'''
# list of possible data items found here: https://et.water.ca.gov/Rest/Index

dir = "./CIMIS/CIMIS_Project_Data/CIMIS_daily_raw_data/"

station_ids = []

for id in range(13):
  rest_url = 'http://et.water.ca.gov/api/data?appKey=46c39c6d-4d71-464c-a20f-a5c7d21bba8e&targets={}&startDate=2024-01-01&endDate=2024-06-06&dataItems=day-eto,day-precip,day-sol-rad-avg,day-vap-pres-avg,day-air-tmp-max,day-air-tmp-min,day-air-tmp-avg,day-rel-hum-max,day-rel-hum-min,day-rel-hum-avg,day-dew-pnt,day-wind-spd-avg,day-wind-run,day-soil-tmp-avg&unitOfMeasure=M'.format(id)
  #rest_url = f'{cimis_api}/data?appKey={api_key}&targets={id}&startDate={start_date}&endDate={end_date}&dataItems={data_items}&unitOfMeasure=M'
  res = requests.get(rest_url)
  print(res.text)


  if res.status_code == 200:
    response = json.loads(res.text)
    payload = response['Data']['Providers'][0]['Records']
    if payload != []:
      print(id)
      station_ids.append(id)

      df = pd.json_normalize(payload)
      df = df[df.columns.drop(list(df.filter(regex='Unit')))]
      df = df[df.columns.drop(list(df.filter(regex='Qc')))]
      print(os.getcwd())
      if not os.path.exists(dir):
        os.makedirs(dir)
      filename = f'{dir}station_id{id}_cimis_daily_raw.csv'
      df.to_csv(filename, index=False)
    else:
      continue
  else:
    continue

print(station_ids)



"[ERR2006-INVALID TARGET] Invalid target specified. The target must be either a list of stations, California zip codes, or California coordinates."
{"Data":{"Providers":[{"Name":"cimis","Type":"station","Owner":"water.ca.gov","Records":[]}]}}
{"Data":{"Providers":[{"Name":"cimis","Type":"station","Owner":"water.ca.gov","Records":[{"Date":"2024-01-01","Julian":"1","Station":"2","Standard":"metric","ZipCodes":"93624","Scope":"daily","DayAirTmpAvg":{"Value":"9.2","Qc":" ","Unit":"(C)"},"DayAirTmpMax":{"Value":"14","Qc":" ","Unit":"(C)"},"DayAirTmpMin":{"Value":"6.2","Qc":" ","Unit":"(C)"},"DayDewPnt":{"Value":"8.9","Qc":" ","Unit":"(C)"},"DayEto":{"Value":"0.52","Qc":" ","Unit":"(mm)"},"DayPrecip":{"Value":"0.1","Qc":" ","Unit":"(mm)"},"DayRelHumAvg":{"Value":"98","Qc":" ","Unit":"(%)"},"DayRelHumMax":{"Value":"100","Qc":" ","Unit":"(%)"},"DayRelHumMin":{"Value":"83","Qc":" ","Unit":"(%)"},"DaySoilTmpAvg":{"Value":"13.9","Qc":"Y","Unit":"(C)"},"DaySolRadAvg":{"Value":"60","Qc":" ","Unit":